In [1]:
import numpy as np

In [2]:
# given generator matrix (G), parity check matrix (H) and decoder matrix (R)
GENERATOR_G = np.array([
                        [1,1,0,1],
                        [1,0,1,1],
                        [1,0,0,0],
                        [0,1,1,1],
                        [0,1,0,0],
                        [0,0,1,0],
                        [0,0,0,1]]) 

PARITY_H = np.array([[1,0,1,0,1,0,1],
                      [0,1,1,0,0,1,1],
                      [0,0,0,1,1,1,1]])

DECODE_R = np.array([[0,0,1,0,0,0,0],
              [0,0,0,0,1,0,0],
              [0,0,0,0,0,1,0],
              [0,0,0,0,0,0,1]])

In [3]:
def encode_hamming(w):
    """takes 4-bit message w and returns 7-bit hamming codeword 
    by multiplying w with the generator matrix"""
    # input message has to be a numpy array, otherwise TypeError will be raised
    if type(w) != np.ndarray:
        raise TypeError("Input argument has to be a numpy array.")
    # will only encode 4-bit messages, other sizes will lead to a ValueError 
    elif np.shape(w) != (4,):
        raise ValueError("Word input w is not 4 bits long") 
    # ValueError if non-binary input
    if not np.isin(w, [0, 1]).all():
        raise ValueError("Input argument is not boolean.")
    # multiplies the input message with the generator matrix
    else: 
        i = np.dot(GENERATOR_G,w)
        c_w = np.mod(i,2)
        # returns encoded 7-bit binary codeword 
        return c_w

In [4]:
def paritycheck_hamming(c):
    """Takes hamming 7-bit codeword and performs paritiy check
    by multiplying the codeword with the parity-check matrix. 
    Returns True when parity check was successfull(zero vector) 
    or False when unsuccessfull(other than 0 vector)"""
    # TypeError if input message not numpy array
    if type(c) != np.ndarray:
        raise TypeError("Input argument has to be a numpy array.")
    # ValueError if input message not 7-bit long
    elif np.shape(c) != (7,):
        raise ValueError("Word input w is not 7 bits long") 
    # ValueError if non-binary input
    if not np.isin(c, [0, 1]).all():
        raise ValueError("Array does not consist of binary values / Input argument is not boolean.")
    # parity matrix H and codeword are multiplied
    #modulo2 solution of the dot product determines whether zero vector or not
    else:
        dot_parity = np.dot(PARITY_H, c)
        mod_parity = np.mod(dot_parity, 2)
        # if no other values than zero are present the transmission does not contain errors
        if not np.any(mod_parity):
            return True
        # if other values than zero are present the transmission has errors
        else:
            return False

In [5]:
def decode_hamming(c_w):
    """decodes given 7-bit vector into original 4-bit vector word"""
    # function only decodes numpy arrays
    if type(c_w) != np.ndarray:
        raise TypeError("Input argument has to be a numpy array.")
    # input argument has to be 7 bits long
    elif np.shape(c_w) != (7,):
        raise ValueError("Word input w is not 7 bits long") 
    # raises ValueError for arrays containing other numbers booleans
    if not np.isin(c_w, [0, 1]).all():
        raise ValueError("Input argument is not boolean.")
    # calculates the dot product of decoder matrix and 7-bit codeword
    # modulo2 results in the original 4-bit input message
    else:
        w = np.dot(DECODE_R, c_w)
        wmod = np.mod(w, 2)
    return wmod


In [6]:
# testing code

import random
for i in range(10):
    #generate random 4-bit vectors to test functionalities
    random_vector = np.random.randint(2, size=4)
    #create encoded hamming word
    random_encoded_vector = encode_hamming(random_vector)
    parity_check_with_random = paritycheck_hamming(random_encoded_vector)
    decoded_vector_random = decode_hamming(random_encoded_vector)
    print(f"Random vector {i}:\t\t {random_vector}")
    print(f"Random encoded vector {i}:\t {random_encoded_vector}")
    print(f"Decoded vector {i}:\t\t {decoded_vector_random}")
    print(f"Parity-check {i} sucessfull?:\t {parity_check_with_random}\n")    

Random vector 0:		 [0 1 0 1]
Random encoded vector 0:	 [0 1 0 0 1 0 1]
Decoded vector 0:		 [0 1 0 1]
Parity-check 0 sucessfull?:	 True

Random vector 1:		 [0 0 0 0]
Random encoded vector 1:	 [0 0 0 0 0 0 0]
Decoded vector 1:		 [0 0 0 0]
Parity-check 1 sucessfull?:	 True

Random vector 2:		 [0 1 1 1]
Random encoded vector 2:	 [0 0 0 1 1 1 1]
Decoded vector 2:		 [0 1 1 1]
Parity-check 2 sucessfull?:	 True

Random vector 3:		 [1 1 0 0]
Random encoded vector 3:	 [0 1 1 1 1 0 0]
Decoded vector 3:		 [1 1 0 0]
Parity-check 3 sucessfull?:	 True

Random vector 4:		 [1 0 1 0]
Random encoded vector 4:	 [1 0 1 1 0 1 0]
Decoded vector 4:		 [1 0 1 0]
Parity-check 4 sucessfull?:	 True

Random vector 5:		 [1 0 0 1]
Random encoded vector 5:	 [0 0 1 1 0 0 1]
Decoded vector 5:		 [1 0 0 1]
Parity-check 5 sucessfull?:	 True

Random vector 6:		 [1 0 1 1]
Random encoded vector 6:	 [0 1 1 0 0 1 1]
Decoded vector 6:		 [1 0 1 1]
Parity-check 6 sucessfull?:	 True

Random vector 7:		 [0 1 0 0]
Random encoded vect

In [7]:
def sketchy_transmission(encoded_message):
    """takes message as input and changes random bits from 0 to 1 
    and vice versa returning the new potentially changed message"""
    after_transfer = np.copy(encoded_message)
    errors_to_inject = random.randint(1,6) 
    bits_to_flip = random.randint(0, errors_to_inject)
    if bits_to_flip == 0:
        return encoded_message
    for f in range(bits_to_flip): 
        # chooses position for each bit to flip
        rand_pos = random.randint(0,6) 
        # flip bit
        after_transfer[rand_pos] = not after_transfer[rand_pos]
    return after_transfer

        
# generate "fake" codewords
for i in range(10000):
    # generate random encoded word
    random_word = np.random.randint(2, size=4)
    random_encoded_w = encode_hamming(random_word)
    # maybe flip some bits
    potential_error_w = sketchy_transmission(random_encoded_w)
    message_valid = paritycheck_hamming(potential_error_w)
    # if parity check returns true but bits were flipped 
    # supposedly valid messages are printed
    if (message_valid) and not np.array_equal(random_encoded_w,potential_error_w):
        print(f"Initial word:\t\t\t{random_encoded_w}")
        print(f"Message supposedly valid:\t{potential_error_w}\n")
        


Initial word:			[0 0 1 1 0 0 1]
Message supposedly valid:	[1 0 1 0 1 0 1]

Initial word:			[0 0 1 0 1 1 0]
Message supposedly valid:	[1 1 1 1 1 1 1]

Initial word:			[0 0 1 1 0 0 1]
Message supposedly valid:	[1 0 1 0 1 0 1]

Initial word:			[1 1 1 1 1 1 1]
Message supposedly valid:	[0 0 1 0 1 1 0]

Initial word:			[1 0 1 0 1 0 1]
Message supposedly valid:	[0 0 1 1 0 0 1]

Initial word:			[0 0 0 1 1 1 1]
Message supposedly valid:	[0 1 0 1 0 1 0]

Initial word:			[1 0 1 1 0 1 0]
Message supposedly valid:	[0 1 1 0 0 1 1]

Initial word:			[1 0 0 0 0 1 1]
Message supposedly valid:	[1 0 1 0 1 0 1]

Initial word:			[0 0 0 1 1 1 1]
Message supposedly valid:	[0 1 1 0 0 1 1]

Initial word:			[0 1 0 0 1 0 1]
Message supposedly valid:	[1 1 0 1 0 0 1]

Initial word:			[0 1 0 1 0 1 0]
Message supposedly valid:	[1 1 1 0 0 0 0]

Initial word:			[0 1 1 1 1 0 0]
Message supposedly valid:	[1 1 0 1 0 0 1]

Initial word:			[1 1 0 0 1 1 0]
Message supposedly valid:	[1 1 1 0 0 0 0]

Initial word:			[1 0 1 0 

Initial word:			[1 0 1 1 0 1 0]
Message supposedly valid:	[1 1 0 0 1 1 0]

Initial word:			[1 1 1 1 1 1 1]
Message supposedly valid:	[1 1 1 0 0 0 0]

Initial word:			[0 1 1 1 1 0 0]
Message supposedly valid:	[0 1 0 0 1 0 1]

Initial word:			[1 0 1 1 0 1 0]
Message supposedly valid:	[1 1 0 0 1 1 0]

Initial word:			[0 1 0 0 1 0 1]
Message supposedly valid:	[0 1 1 1 1 0 0]

Initial word:			[0 1 0 1 0 1 0]
Message supposedly valid:	[1 1 0 0 1 1 0]

Initial word:			[1 1 1 1 1 1 1]
Message supposedly valid:	[0 0 0 1 1 1 1]

Initial word:			[1 0 0 1 1 0 0]
Message supposedly valid:	[1 0 1 1 0 1 0]

Initial word:			[0 0 1 0 1 1 0]
Message supposedly valid:	[1 1 1 0 0 0 0]

Initial word:			[1 0 1 1 0 1 0]
Message supposedly valid:	[1 1 1 0 0 0 0]

Initial word:			[1 0 0 0 0 1 1]
Message supposedly valid:	[1 1 0 1 0 0 1]

Initial word:			[0 1 0 1 0 1 0]
Message supposedly valid:	[1 1 0 0 1 1 0]

Initial word:			[1 0 1 0 1 0 1]
Message supposedly valid:	[0 1 0 0 1 0 1]

Initial word:			[1 1 1 0 

Initial word:			[0 1 1 0 0 1 1]
Message supposedly valid:	[1 1 1 0 0 0 0]

Initial word:			[0 1 1 1 1 0 0]
Message supposedly valid:	[1 1 0 1 0 0 1]

Initial word:			[0 1 0 0 1 0 1]
Message supposedly valid:	[1 1 0 1 0 0 1]

Initial word:			[1 0 1 1 0 1 0]
Message supposedly valid:	[0 1 1 0 0 1 1]

Initial word:			[0 1 0 1 0 1 0]
Message supposedly valid:	[0 0 1 1 0 0 1]

Initial word:			[0 1 0 0 1 0 1]
Message supposedly valid:	[1 0 0 1 1 0 0]

Initial word:			[0 1 1 0 0 1 1]
Message supposedly valid:	[1 1 1 0 0 0 0]

Initial word:			[1 0 0 1 1 0 0]
Message supposedly valid:	[1 0 1 0 1 0 1]

Initial word:			[1 1 1 0 0 0 0]
Message supposedly valid:	[1 0 0 1 1 0 0]

Initial word:			[0 0 1 1 0 0 1]
Message supposedly valid:	[1 1 1 1 1 1 1]

Initial word:			[0 0 1 1 0 0 1]
Message supposedly valid:	[1 0 0 0 0 1 1]

Initial word:			[1 1 1 0 0 0 0]
Message supposedly valid:	[1 1 0 1 0 0 1]

Initial word:			[1 1 1 0 0 0 0]
Message supposedly valid:	[1 0 1 0 1 0 1]

Initial word:			[0 0 0 0 

In [8]:
# testing errors 

# encode method
encode_noarray = [1,0,1]
encode_hamming(encode_noarray)

TypeError: Input argument has to be a numpy array.

In [ ]:
# wrong length of input
encode_wronglength = np.array([1,0,1])
encode_hamming(encode_wronglength)


In [9]:
# non binary input
encode_nonbinary = np.array([1,2,3,4])
encode_hamming(encode_nonbinary)


ValueError: Input argument is not boolean.

In [10]:
# partiy check method
# no numpy array as input
parity_noarray = [1,0,1]
paritycheck_hamming(parity_noarray)

TypeError: Input argument has to be a numpy array.

In [11]:
# other than 7-bit lengh
parity_wronglength = np.array([1,0,1])
paritycheck_hamming(parity_wronglength)

ValueError: Word input w is not 7 bits long

In [12]:
# non binary input
parity_nonbinary = np.array([1,0,1,3,4,5,6])
paritycheck_hamming(parity_nonbinary)

ValueError: Array does not consist of binary values / Input argument is not boolean.

In [13]:
# decode method
# no numpy array as input
decode_noarray = [1,0,1,0,1,1,1]
decode_hamming(decode_noarray)

TypeError: Input argument has to be a numpy array.

In [14]:
decode_wronglength = np.array([1,0,1,0,1,1,1,1])
decode_hamming(decode_wronglength)

ValueError: Word input w is not 7 bits long

In [15]:
decode_nonbinary = np.array([1,0,1,0,1,1,3])
decode_hamming(decode_nonbinary)

ValueError: Input argument is not boolean.